In [ ]:
import glob
import numpy as np
import re
import matplotlib.pyplot as plt

interfaces = [ 1.9199, 2.1319, 2.1999, 2.2599,
               2.2799, 2.2999, 2.3099, 2.3399,
               2.3799, 2.3999, 2.4199, 2.4299, 2.4999,
               2.5999, 4.0000]

ninterfaces = len(interfaces)-1
bin_width = 0.01 
max_bins = int((interfaces[-1]-interfaces[0])/bin_width)                        # number of lambdas between the first and last interface
counts = np.zeros((ninterfaces,max_bins))                                       # matrix to store crossing counts (for each lambda between the first and last interface)
                                                                                # size: interfaces x max_bins 

for i in range(ninterfaces):
    ndiscard = 10                                                               # this corresponds to the number of paths between each decorrelated sample/path
    start = 600
    n_moves = 1000
    for move in range(start,n_moves,ndiscard):
        
        filen = f"../../{i}/paths/path_gl-{move}_ens-{i}.txt"

        path = np.genfromtxt(filen,comments=None,dtype=[('pathid','U50'),('time','f8'),('op','f8')],encoding='utf8') 
        
        val = np.max(path['op'])                                                # gets the max op of this path, tells you how far this path has travelled        
        
        nbin = int((val-interfaces[i])/bin_width)                               # tells us how many lambdas has this path crosses

        for j in range(min(nbin+1,max_bins)):
            counts[i][j] += 1                                                   # adds 1 for each lambda that this path crosses

# Get crossing probability histogram 

lambdas = [[] for i in range(ninterfaces)]
crossingprob = [[] for i in range(ninterfaces)]

for i in range(ninterfaces):
    for j in range(max_bins):
        if counts[i][j] > 0:
            lambdas[i].append(interfaces[i]+j*bin_width)                        # this tracks the lambda corresponding to the crossing probability at that lambda
            crossingprob[i].append(counts[i][j]/counts[i][0])                   # P(lambda|lambda_i), counts[i][0] is the total number of paths of that ensemble because 
                                                                                # all paths from this ensemble would have crossed that first interface
    
# Plot the histogram

for i in range(ninterfaces):
    plt.plot(lambdas[i],crossingprob[i], label='Interface {}'.format(i))
    plt.plot(np.linspace(interfaces[i],interfaces[i],10),np.linspace(0,1,10), color='grey')
    plt.xlabel('OP')
    plt.ylabel('$P(\u03BB|\u03BB_{i})$')
# plt.legend()
plt.xlim(interfaces[0],interfaces[-1])
plt.show() 
